In [1]:
from dotenv import load_dotenv
from conversationService import get_embedding
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import loggingService
import os
import weaviate

load_dotenv()
logger = loggingService.get_logger()


/Users/vanildovanni/miniconda3/envs/ml/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/Users/vanildovanni/miniconda3/envs/ml/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/Users/vanildovanni/miniconda3/envs/ml/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
sentence-transformers/gtr-t5-large
2023-09-26 11:23:54,952 - numexpr.utils - INFO - Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-09-26 11:23:54,952 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [2]:
apikey = os.getenv("GEANAI_KEY", None)
class_name = os.getenv("WEVIATE_CLASS", 'Livros')
path = os.getenv("DATA_PATH", "data")
weaviate_url = os.getenv("WEAVIATE_URL", 'http://127.0.0.1:8080')

client = weaviate.Client(
    url=weaviate_url,
)

pages = []
pdf_loader = PyPDFDirectoryLoader(path)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50, separators=["\n\n", "\n", " ", "", "\n \n", "\n \n \n","\n\n\n"])
documents = pdf_loader.load_and_split(text_splitter=text_splitter)
print(len(documents))
print(documents[0])

2199
page_content='Memórias Póstumas de Brás Cubas, Machado de Assis \n \nFonte: \nAssis, Machado de. Obra Completa. vol. I. Rio de Janeiro: Nova Aguilar, 1994. \n \nTexto proveniente de: \nA Biblioteca Virtual do Estudante Brasileiro <http://www.bibvirt.futuro.usp.br> \nA Escola do Futuro da Universidade de São Paulo \nPermitido o uso apenas para fins educacionais. \n \nTexto-base digitalizado por: \nNúcleo de Pesquisas em Informática, Literatura e Lingüística \n<http://www.cce.ufsc.br/~nupill/literatura/literat.html> \n \nEste material pode ser redistribuído livremente, desde que não seja alterado, e que as informações \nacima sejam mantidas. Para maiores informações, escreva para <bibvirt@futuro.usp.br>. \n \nEstamos em busca de patrocinadores e voluntários para nos ajudar a manter este projeto. Se você \nquer ajudar de alguma forma, mande um e-mail para <bibvirt@futuro.usp.br> e saiba como isso é \npossível. \n \n \n \nMEMÓRIAS PÓSTUMAS DE BRÁS CUBAS \nMACHADO DE ASSIS \n \n \n    

In [ ]:
def pdf_text_splitter(pdf_text) -> dict:
  retorno = {'content': '', 'source': '', 'page': 0}
  
  retorno['content'] = getattr(pdf_text, 'page_content')
  retorno['souce'] = getattr(pdf_text, 'metadata')['source']
  retorno['page'] = getattr(pdf_text, 'metadata')['page']
  
  return retorno

for doc in documents:
  # logger.debug(pdf_text_splitter(doc))
  # print(pdf_text_splitter(doc))
  pages.append(pdf_text_splitter(doc))
  
print(pages[0])

In [ ]:
class_document =  {
  "class": "Livros",
  "description": "Dados do documento PDF",
  "moduleConfig": {
    "text2vec-transformers": {
      "poolingStrategy": "masked_mean",
      "vectorizeClassName": False
    }
  },
  "properties": [
    {
      "dataType": [
        "text"
      ],
      "description": "Conteúdo do documento PDF",
      "moduleConfig": {
        "text2vec-transformers": {
          "skip": False,
          "vectorizePropertyName": False
        }
      },
      "name": "content"
    },
    {
      "dataType": [
        "text"
      ],
      "description": "",
      "moduleConfig": {
        "text2vec-transformers": {
          "skip": False,
          "vectorizePropertyName": False
        }
      },
      "name": "source"
    },
    {
      "dataType": [
        "text"
      ],
      "description": "",
      "moduleConfig": {
        "text2vec-transformers": {
          "skip": False,
          "vectorizePropertyName": False
        }
      },
      "name": "page"
    }
  ],
  "vectorizer": "none"
}

client.schema.create_class(class_document)
# client.schema.property.create(class_name, {
#     'name': 'exemplar',
#     'dataType': ['text'],
# })

# client.schema.delete_class('Livros')
client.schema.get("Livros")

In [ ]:
with client.batch as batch:
  batch.batch_size=100
  i = 0
    
  for page in pages:
    logger.info(f"importing question: {i+1}")
    i = i+1
    
    properties = {
            "content": page["content"],
            "page": str(page["page"]),
            "source": page["source"],
        }
    
    vector = get_embedding(page["content"])

    client.batch.add_data_object(properties, 'Livros', vector=vector)

In [3]:
result = (client.query
.get('Livros', ["content", "source", "page"])
.with_additional(["certainty", "distance"]) # note that certainty is only supported if distance==cosine
.with_near_vector({
  "vector": get_embedding('o que significa casmurro'),
  "certainty": 0.8
})
.with_limit(4)
.do()
)

print(result['data']['Get'][class_name])

retorno = ''
if len(result['data']['Get'][class_name]) > 0:
  retorno = result['data']['Get'][class_name][0]['content']

  for contexto in result['data']['Get'][class_name][1:]:
    retorno += f"\n{contexto['content']}"

logger.info(retorno)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[{'_additional': {'certainty': 0.881557047367096, 'distance': 0.2368859}, 'content': 'pegou. N em por i sso me zangue i. Cont ei a anedota aos amigos da cidade, e eles, por gra ça, \nchamam-me assim, alguns  em bilhetes: "Dom Casmurro, dom ingo vou j antar com você."--"V ou \npara Petrópol is, Dom Casmurro;  a casa é a mesma da Renania; vê se deixas essa caverna do \nEngenho N ovo, e  vai lá passar uns  quinze dias comigo."--"M eu caro Dom Casmurro, nã o cuide que \no dispenso do t eatro amanhã; venha e dorm irá aqui na cidade; dou-l he camarote, dou-l he chá, dou-\nlhe cama; só não lhe dou m oça." \nNão consultes dicionários. Casmurro nã o está aqui no s entido que  eles lhe dão, mas no que  lhe pôs \no vul go de  hom em calado e metido consigo. D om veio por i ronia, para atribuir-me fumos de \nfidalgo. T udo por e star cochilando! T ambém não achei melhor título para a minha narração - se não \ntiver outro da qui até ao fim do livro, va i este mesmo. O meu poe ta do trem ficará sabe